In [1]:
import os
import sys
import json

sys.path.insert(0, '../../../pipelines/tasks/mlb/')

from common.helpers.parsers import EventDescriptionParser

In [3]:
players = set()
ats = set()
event_types = set()

bad_ats = []
bad_types = []
bad_players = ['catcher']

replay = False
parser = EventDescriptionParser()

for file in os.listdir('../../../data/mlb/pbp/'):

    with open(f'../../../data/mlb/pbp/{file}', 'r', encoding='UTF8') as pbp_input:
        game = json.load(pbp_input)

    for period in game['periods']:
        for event in period['events']:

            if replay:
                event['entities'] = parser.transform_into_object(event['desc'])

            entities = event['entities'] if 'entities' in event else {}

            if 'player' in entities:
                players.add(entities['player'])

                if entities['player'] in bad_players:
                    print(entities)
                    print(event)
                    print(file)
                    print()

            if 'at' in entities:
                ats.add(entities['at'])

                if entities['at'] in bad_ats:
                    print(entities)
                    print(event)
                    print(file)
                    print()

            if 'type' in entities:
                event_types.add(entities['type'])

                if entities['type'] in bad_types:
                    print(entities)
                    print(event)
                    print(file)
                    print()

            if 'moves' in entities:
                for move in entities['moves']:
                    if 'player' in move:
                        players.add(move['player'])

                        if move['player'] in bad_players:
                            print(entities)
                            print(event)
                            print(file)
                            print()

                    if 'at' in move:
                        ats.add(move['at'])

                        if move['at'] in bad_ats:
                            print(entities)
                            print(event)
                            print(file)
                            print()

                    if 'type' in move:
                        event_types.add(move['type'])

                        if move['type'] in bad_types:
                            print(entities)
                            print(event)
                            print(file)
                            print()

In [4]:
sorted(event_types, key=len, reverse=True)

["grounded into fielder's choice",
 'grounded into double play',
 "catcher's interference",
 'intentionally walked',
 'ground rule double',
 'caught stealing',
 'infield single',
 'fielding error',
 'throwing error',
 'sacrifice fly',
 'grounded out',
 'hit by pitch',
 'bunt single',
 'wild pitch',
 'struck out',
 'picked off',
 'fouled out',
 'popped out',
 'lined out',
 'flied out',
 'advanced',
 'singled',
 'doubled',
 'homered',
 'tripled',
 'walked',
 'sub-f',
 'stole',
 'sub-p',
 'out']

In [5]:
sorted(ats, key=len, reverse=True)

['shallow right center',
 'shallow left center',
 'deep right center',
 'deep left center',
 'home in rundown',
 'shallow center',
 'right center',
 'shallow left',
 'left center',
 'deep center',
 'deep right',
 'deep left',
 'shortstop',
 'pitcher',
 'catcher',
 'second',
 'center',
 'third',
 'right',
 'first',
 'home',
 'left']

In [12]:
sorted(players, key=len, reverse=True)

['Hirokazu Sawamura',
 'D. Steckenrider',
 'Caleb Thielbar',
 'G. Cleavinger',
 'A. Misiewicz',
 'J. Sheffield',
 'D. Rodriguez',
 'B. Graterol',
 'G. Whitlock',
 'Bradley Jr.',
 'E. Phillips',
 'C. Thielbar',
 'K. Crawford',
 'H. Sawamura',
 'D. Castillo',
 'D. Coulombe',
 'M. Gonzales',
 'L. Gilbert',
 'Y. Ramirez',
 'J. Diekman',
 'C. Kershaw',
 'E. Swanson',
 'N. Pivetta',
 'C. Paddack',
 'C. Stashak',
 'R. Brasier',
 'J. Bruihl',
 'P. Sewald',
 'Kirilloff',
 'J. Turner',
 'C. Archer',
 'Rodríguez',
 'M. Barnes',
 'Bellinger',
 'T. Duffey',
 'A. Heaney',
 'J. Alcala',
 'J. Cotton',
 'J. Romero',
 'C. Flexen',
 'P. Valdez',
 'D. Hudson',
 'T. Turner',
 'Hernández',
 'J. Winder',
 'Celestino',
 'M. Strahm',
 'J. Smith',
 'A. Munoz',
 'D. Price',
 'Bogaerts',
 'A. Vesia',
 'D. Bundy',
 'M. Festa',
 'Sawamura',
 'M. Wacha',
 'Martinez',
 'Plawecki',
 'J. Duran',
 'T. Houck',
 'A. Davis',
 'Crawford',
 'E. Pagan',
 'Vázquez',
 'Larnach',
 'Freeman',
 'Frazier',
 'Torrens',
 'S. Romo',
 

In [6]:
output = []
for player in players:
    for chunk in player.split(' '):
        if chunk[0].islower():
            output.append(player)

sorted(output, key=len, reverse=True)

[]

In [10]:
dump_file = True
if dump_file:

    keys = set()
    text_instances = []

    for file in os.listdir('../../../data/mlb/pbp/'):

        with open(f'../../../data/mlb/pbp/{file}', 'r', encoding='UTF8') as pbp_input:
            game = json.load(pbp_input)

        for period in game['periods']:
            for event in period['events']:

                description = event['desc']
                if not description in keys:
                    text_instances.append((description, event['entities']))
                    keys.add(description)

    with open('../../../pipelines/tasks/mlb/common/tests/docs/desc_to_entities.json', 'w') as test_instances:
        json.dump(text_instances, test_instances, indent=2)